In [2]:
import cv2
import numpy as np
def segmentation():    
    I = cv2.imread(f'pedestrians_empty_bg.jpg')
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cap = cv2.VideoCapture('pedestrians_input.mp4')
    # success, image = vidcap.read()
    YY, XX = I.shape    
    N = 60
    iN = 0  
    BUF = np.zeros((YY, XX, N), np.uint8)

    while True:
        if cap.grab():
            flag, frame = cap.retrieve()
            IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            BUF[:, :, iN] = IG
            
            iN += 1
            if iN >= N:
                iN = 0

            # val = np.mean(BUF, axis=2).astype('uint8')
            val = np.median(BUF, axis=2).astype('uint8')
            scene = cv2.absdiff(val, IG)
            blur = cv2.GaussianBlur(scene, (3, 3), 0)
            ret, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
            # cv2.imshow("I", final_img)
            cv2.imshow("I", thresh)
        cv2.waitKey(10)
    cv2.destroyAllWindows()

segmentation()

KeyboardInterrupt: 

In [7]:
import cv2
import numpy as np


def segmentation_test(folder, metric):    
    I = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    YY, XX = I.shape    
    N = 60
    iN = 0  
    BUF = np.zeros((YY, XX, N), np.uint8)

    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    
    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY)
        BUF[:, :, iN] = IG
            
        iN += 1
        if iN >= N:
            iN = 0

        val = None
        if metric == 'median':
            val = np.median(BUF, axis=2).astype('uint8')
        elif metric == 'mean':
            val = np.mean(BUF, axis=2).astype('uint8')
            
        scene = cv2.absdiff(val, IG)
        blur = cv2.GaussianBlur(scene, (3, 3), 0)
        ret, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        B = thresh

        I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP_M = np.logical_and((B == 255), (GTB == 255)) 
        TP_S = np.sum ( TP_M ) 
        TP = TP + TP_S 

        TN_M = np.logical_and((B == 0), (GTB == 0)) 
        TN_S = np.sum (TN_M) 
        TN += + TN_S 

        FP_M = np.logical_and((B == 255), (GTB == 0))
        FP_S = np.sum(FP_M)
        FP += FP_S

        FN_M = np.logical_and((B == 0), (GTB == 255))
        FN_S = np.sum(FN_M)
        FN += FN_S

        cv2.imshow("I", B)
        cv2.waitKey(10)
    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nType: {metric}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

segmentation_test('pedestrian', 'mean')

Clip Name: pedestrian
Type: mean
Precission: 0.10038338260719117 Recall: 0.9478775068493968 F1: 0.18154097208403766


In [8]:
segmentation_test('office', 'mean')
segmentation_test('office', 'median')

Clip Name: office
Type: mean
Precission: 0.34205092103981477 Recall: 0.35431910368019603 F1: 0.34807694603035666
Clip Name: office
Type: median
Precission: 0.4129851378809777 Recall: 0.2996011151926836 F1: 0.3472725086498542


In [13]:
import cv2
import numpy as np


def segmentation_approx(folder, type, alfa):    
    BG = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    BG = cv2.cvtColor(BG, cv2.COLOR_BGR2GRAY).astype('float64')

    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    
    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('float64')

        BGn = None
        if type == 'mean':
            BGn = alfa*IG + (1-alfa)*BG
        elif type == 'median':
            if np.all(BG) < np.all(IG):
                BGn = BG + 1
            elif np.all(BG) > np.all(IG):
                BGn = BG - 1
            else:
                BGn = BG
    
            
        scene = cv2.absdiff(BGn, IG)
        blur = cv2.GaussianBlur(scene, (3, 3), 0)
        ret, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        B = thresh

        I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP_M = np.logical_and((B == 255), (GTB == 255)) 
        TP_S = np.sum ( TP_M ) 
        TP = TP + TP_S 

        TN_M = np.logical_and((B == 0), (GTB == 0)) 
        TN_S = np.sum (TN_M) 
        TN += + TN_S 

        FP_M = np.logical_and((B == 255), (GTB == 0))
        FP_S = np.sum(FP_M)
        FP += FP_S

        FN_M = np.logical_and((B == 0), (GTB == 255))
        FN_S = np.sum(FN_M)
        FN += FN_S

        cv2.imshow("I", B)
        cv2.waitKey(10)

        BG = BGn    

    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nType: {type}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

In [15]:
segmentation_approx('office', 'mean', 0.01)
segmentation_approx('office', 'median', 0.01)

Clip Name: office
Type: mean
Precission: 0.5396295428284262 Recall: 0.535529352029728 F1: 0.5375716292523098
Clip Name: office
Type: median
Precission: 0.27435928752508715 Recall: 0.9617518624537063 F1: 0.4269285262304305


In [1]:
import cv2
import numpy as np


def segmentation_conservatice(folder, type, alfa):    
    BG = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    BG = cv2.cvtColor(BG, cv2.COLOR_BGR2GRAY).astype('float64')
    BG_last = BG
    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    
    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('float64')

        BGn = BG
        valid = (BG == BG_last)

        if type == 'mean':
            BGn[valid] = alfa*IG[valid] + (1-alfa)*BG[valid]

        elif type == 'median':
            if np.all(BG) < np.all(IG):
                BGn[valid] = BG[valid] + 1
            elif np.all(BG) > np.all(IG):
                BGn[valid] = BG[valid] - 1
            else:
                BGn[valid] = BG[valid]
    
        scene = cv2.absdiff(BGn, IG)
        blur = cv2.GaussianBlur(scene, (3, 3), 0)
        ret, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        B = thresh

        I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP_M = np.logical_and((B == 255), (GTB == 255)) 
        TP_S = np.sum ( TP_M ) 
        TP = TP + TP_S 

        TN_M = np.logical_and((B == 0), (GTB == 0)) 
        TN_S = np.sum (TN_M) 
        TN += + TN_S 

        FP_M = np.logical_and((B == 255), (GTB == 0))
        FP_S = np.sum(FP_M)
        FP += FP_S

        FN_M = np.logical_and((B == 0), (GTB == 255))
        FN_S = np.sum(FN_M)
        FN += FN_S

        cv2.imshow("I", B)
        cv2.waitKey(10)

        BG_last = BG
        BG = BGn    

    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nType: {type}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

In [2]:
segmentation_conservatice('office', 'mean', 0.01)

Clip Name: office
Type: mean
Precission: 0.5396295428284262 Recall: 0.535529352029728 F1: 0.5375716292523098


In [40]:
import cv2
import numpy as np


def segmentation_OpenCV(folder, method):    
    BG = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    BG = cv2.cvtColor(BG, cv2.COLOR_BGR2GRAY).astype('float64')
    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)

    mog = None
    if method == "MOG2":
        mog = cv2.createBackgroundSubtractorMOG2(history=300,
                                                varThreshold=200,
                                                detectShadows=False)
    elif method == "KNN":
        mog = cv2.createBackgroundSubtractorKNN()

    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('float64')

        B = mog.apply(IG)

        I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP_M = np.logical_and((B == 255), (GTB == 255)) 
        TP_S = np.sum ( TP_M ) 
        TP = TP + TP_S 

        TN_M = np.logical_and((B == 0), (GTB == 0)) 
        TN_S = np.sum (TN_M) 
        TN += + TN_S 

        FP_M = np.logical_and((B == 255), (GTB == 0))
        FP_S = np.sum(FP_M)
        FP += FP_S

        FN_M = np.logical_and((B == 0), (GTB == 255))
        FN_S = np.sum(FN_M)
        FN += FN_S

        cv2.imshow("I", B)
        cv2.waitKey(10)

    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

# segmentation_OpenCV('pedestrian', 'MOG2')

In [42]:
# segmentation_OpenCV('pedestrian', 'MOG2')
segmentation_OpenCV('pedestrian', 'KNN')

KeyboardInterrupt: 